In [15]:
import import_ipynb
from os import walk, stat, listdir, path, environ, remove
from shutil import copy
import pandas as pd
from rdflib import Graph
from codes.prep_onto import prep_onto
from codes.prompt_API import create_news, create_newsdict, generate_onto
from UnifiedOntologyPipeline.pipeline.appendOnto import append_file, process_lines
from UnifiedOntologyPipeline.pipeline.connectIndividuals import getNamespace, connectIndividuals, clear_serialize

### **Global Variables**

In [2]:
try:
    from dotenv import load_dotenv
    load_dotenv("./codes/.env")
except ImportError:
    print("dotenv not installed, skipping...")

In [3]:
TOKEN = environ.get("APIKEY")
URL = "https://api.deepseek.com"
MODEL = "deepseek-chat"

### **News Preparation**

In [4]:
news_path = "./codes/TheHackerNews_Dataset.xlsx"
df = pd.read_excel(news_path)['Article']
df.shape

(3742,)

In [5]:
%%capture
news_path = "./0_newsInput/"
create_news(df, output_dir=news_path)

In [6]:
news_list = []
for file in listdir(news_path):
    if file.endswith(".txt"):
        news_list.append(file)
news_list.sort()

In [7]:
chosen_news_list = news_list[10:15]

In [8]:
news_dict = create_newsdict(news_list=chosen_news_list, news_path=news_path)
news_dict

{'news011.txt': 'A newly discovered glitch in Zoom\'s screen sharing feature can accidentally leak sensitive information to other attendees in a call, according to the latest findings.\nTracked as CVE-2021-28133, the unpatched security vulnerability makes it possible to reveal contents of applications that are not shared, but only briefly, thereby making it harder to exploit it in the wild.\nIt\'s worth pointing out that the screen sharing functionality in Zoom lets users share an entire desktop or phone screen, or limit sharing to one or more specific applications, or a portion of a screen. The issue stems from the fact that a second application that\'s overlayed on top of an already shared application can reveal its contents for a short period of time.\n"When a Zoom user shares a specific application window via the \'share screen\' functionality, other meeting participants can briefly see contents of other application windows which were not explicitly shared," SySS researchers Michae

### **Input Ontology**

In [9]:
refine_onto_path = "./4_refineOutput/RefinedUnifiedOntology.ttl"
filtered_onto_path = "./5_filteredOnto/FilteredUnifiedOntology.ttl"

if path.exists(refine_onto_path):
    prep_onto(refine_onto_path, filtered_onto_path)
    with open(filtered_onto_path, 'r') as file:
        initial_ontology = file.read()
        file.close()
    
else:
    initial_ontology = None

### **Prompts Preparation**

In [10]:
initial_prompt_text = """I will provide you with a news article, and I want to generate an ontology from it. 
Please extract key concepts, relationships, and categories from the article and structure them into an ontology. 
The ontology should be in a structured format of Turtle (.ttl). 
Here is the news content: 
"""

In [11]:
prompt_text = """I will provide you with a news article, and I want to expand upon an existing ontology. 
Please analyze the new article, extract key concepts, relationships, and categories, and integrate them into the existing ontology while maintaining consistency and avoiding redundancy. 
Ensure that new concepts complement the previous ontology rather than duplicating existing ones.
Only include the instances and properties essential for this specific news. 
Here is the news content: 
"""

### **Prompt to LLM**

In [12]:
input_onto_DIR = "./1_ontologiesInput/"
connect_output_DIR = "./2_connectOutput/"

In [19]:
# Clear directories for new inputs


if path.exists(input_onto_DIR):
    for filename in listdir(input_onto_DIR):
        file_path = path.join(input_onto_DIR, filename)
        
        if path.isfile(file_path) and filename != ".gitkeep":
            copy(file_path, "./codes/news_onto/")  
            remove(file_path)  

if path.exists(connect_output_DIR):
    for filename in listdir(connect_output_DIR):
        file_path = path.join(connect_output_DIR, filename)
        
        if path.isfile(file_path) and filename != ".gitkeep":
            remove(file_path)  

In [14]:
ontoDict, responseList, promptList = generate_onto(URL, TOKEN, MODEL, 
                                                   initial_prompt_text=initial_prompt_text, 
                                                   prompt_text=prompt_text, 
                                                   news_dict=news_dict,
                                                   initial_ontology=initial_ontology,
                                                   output_dir=input_onto_DIR)  

Saved: ./1_ontologiesInput/news011.ttl
Saved: ./1_ontologiesInput/news012.ttl
Saved: ./1_ontologiesInput/news013.ttl
Saved: ./1_ontologiesInput/news014.ttl
Saved: ./1_ontologiesInput/news015.ttl


### **Ontology Merge Pipeline**

In [20]:
def getFilenames(input_path, extension=''):
    return [f for f in next(walk(input_path), (None, None, []))[2] if f.endswith('.ttl')]  # [] if no file

In [21]:
merge_output_DIR = "./3_mergeOutput/"
refine_output_DIR = "./4_refineOutput/"
input = getFilenames(input_onto_DIR, '.ttl')

In [22]:
"""
Every files in {input_path} will be updated with a new class,
and an individual from said class.
The class is named "CyberNews", and the individual is named after the file's name.
Every other individuals in the ontology will have object property of "relatedTo" to said individual. 
"""

connected = []
news_class = "CybersecurityNewsArticle"

for ind, file in enumerate(input):
    if stat(input_onto_DIR + file).st_size == 0: # skip empty files
        continue
    g = Graph()
    process_lines(f"{input_onto_DIR}{file}", f"{input_onto_DIR}{file}")
    g.parse(f"{input_onto_DIR}{file}", format="ttl")

    prefix, uri = getNamespace(g)
    news_name = file.split('.')[0]  # Get filename, exclude extension
    connectIndividuals(g, prefix, uri, news_name, news_class)   

    OutputDes = f"{connect_output_DIR}{news_name}_connected.ttl"  # Get new files' names
    connected.append(OutputDes)
    clear_serialize(g, OutputDes)

In [23]:
merged_file = f"./{merge_output_DIR}UnifiedOntology.ttl"

In [24]:
for ind, connected_file in enumerate(connected):
    append_file(source_file=connected_file, target_file=merged_file)

In [25]:
g = Graph()
g.parse(merged_file, format="ttl")
clear_serialize(g, merged_file)

### **Ontology Refinement Pipeline**

In [26]:
from UnifiedOntologyPipeline.tool.extractClass import *

In [ ]:
%%capture
loginHuggingFace()

In [ ]:
path_to_onto = "./3_mergeOutput/UnifiedOntology.ttl"
df = getOntoClasses(path_to_onto)

In [ ]:
name_tokenizer = "bert-base-uncased"
name_model = "OhWayTee/bert-taxonomy"
name_pipeline = "text-classification"

classifier = getClassifier(name_tokenizer, name_model, name_pipeline)

In [ ]:
df_pred = getPredictions(classifier, df)

In [ ]:
df_top = sortPredictions(df_pred)

In [ ]:
confidence_threshold = 0.9

In [ ]:
df_superclass = df_top[(df_top['label'] == "LABEL_1") & (df_top['score'] >= confidence_threshold)] # classB is SUBCLASS of classA
df_subclass = df_top[(df_top['label'] == "LABEL_2") & (df_top['score'] >= confidence_threshold)] # classA is SUBCLASS of classB
[df_superclass.shape, df_subclass.shape]

In [ ]:
df_superclass_filtered = df_superclass.sort_values('score', ascending=False).drop_duplicates('classB') # remove duplicate subclasses
df_subclass_filtered = df_subclass.sort_values('score', ascending=False).drop_duplicates('classA')
[df_superclass_filtered.shape, df_subclass_filtered.shape]

In [ ]:
g = Graph()
g.parse(path_to_onto, format='turtle')
prefix, uri = getNamespace(g)
print(prefix, uri)

In [ ]:
for idx, row in df_superclass_filtered.iterrows():
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX {prefix}: <{uri}>
    INSERT{{
        ?subclass rdfs:subClassOf ?superclass .
    }}
    WHERE {{
        ?subclass rdfs:label "{row['classB']}" .
        ?superclass rdfs:label "{row['classA']}" .
    }}
    """
    g.update(query)

In [ ]:
for idx, row in df_subclass_filtered.iterrows():
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX {prefix}: <{uri}>
    INSERT{{
        ?subclass rdfs:subClassOf ?superclass .
    }}
    WHERE {{
        ?subclass rdfs:label "{row['classA']}" .
        ?superclass rdfs:label "{row['classB']}" .
    }}
    """
    g.update(query)

In [ ]:
refine_onto_path = "./4_refineOutput/RefinedUnifiedOntology.ttl"

In [ ]:
clear_serialize(g, refine_onto_path)

### **Filtering Ontology for Prompt**

In [ ]:
from codes.prep_onto import prep_onto

In [ ]:
prep_onto(refine_onto_path, filtered_onto_path)